In [1]:
# Install LangChain Hub, a repository for storing and sharing LangChain prompts and chains
!pip install langchainhub

# Install the LangChain OpenAI integration, which provides easy access to OpenAI models
!pip install langchain-openai

# Install the core LangChain library, which provides tools for building applications with LLMs
!pip install langchain

# Install FAISS (Facebook AI Similarity Search), a library for efficient similarity search and clustering of dense vectors
!pip install faiss-cpu

# Upgrade LangChain Community and install Tavily Python, which provides web search functionalities for LangChain
!pip install -U langchain-community tavily-python

# Install a specific version of Gradio Client (0.2.10), a package for interacting with Gradio interfaces programmatically
!pip install gradio_client==0.2.10

# Install Gradio version 3.38.0, a library for creating web-based UI components for AI models and applications
!pip install gradio==3.38.0

# Install LangChain Community
!pip install langchain_community


In [2]:
# Import KaggleHub, a library for accessing datasets and models from Kaggle
import kagglehub

# Import CSV module for reading and writing CSV files
import csv

# Import pandas for data manipulation and analysis
import pandas as pd

# Import math module for mathematical operations
import math

# Import NumPy for numerical computations and working with arrays
import numpy as np

# Import os module for interacting with the operating system
import os

# Import getpass module for securely handling user input (e.g., passwords)
import getpass

# Import StrOutputParser from LangChain Core to process and parse output strings
from langchain_core.output_parsers import StrOutputParser

# Import Google Drive module for mounting Google Drive (Used in Google Colab)
from google.colab import drive  # Used for mounting Google Drive


In [3]:
# Import the 'tool' decorator from LangChain Core, which allows defining custom tools for use in LangChain workflows
from langchain_core.tools import tool

# Define a custom tool using the @tool decorator
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int  # Computes the product of two integers


In [4]:
# Print the tool's name (metadata automatically assigned by LangChain)
print(multiply.name)  # Expected output: "multiply"

# Print the tool's description (extracted from the function's docstring)
print(multiply.description)  # Expected output: "Multiply two integers together."

# Invoke the tool using a dictionary of input values (LangChain's tool invocation pattern)
result = multiply.invoke({"first_int": 4, "second_int": 5})

# Print the computed result (4 * 5 = 20)
print(result)  # Expected output: 20

multiply
Multiply two integers together.
20


In [5]:
# Import the 'render_text_description' function from LangChain's 'tools.render' module
# This function is used to generate a textual description of the given tools
from langchain.tools.render import render_text_description

# Render a text-based description of the multiply tool
# This helps in understanding how the tool can be used, including its name, input parameters, and description
rendered_tools = render_text_description([multiply])

# Print the rendered description of the multiply tool
print(rendered_tools)
# Expected output: A structured textual representation of the tool's name, description, and expected inputs/outputs.


multiply(first_int: int, second_int: int) -> int - Multiply two integers together.


In [9]:
# Import 'ChatPromptTemplate' from 'LangChain Core'
# This is used to create structured prompts for chat-based interactions with LLMs
from langchain_core.prompts import ChatPromptTemplate

# Define a system prompt string with tool details
# The assistant is informed about the available tool ('rendered_tools')
# The assistant is also instructed to return the tool name and arguments in a JSON format when given a user query
system_prompt = f"""You are an assistant with the following tool:

{rendered_tools}

Given a user query, return the tool name and input values in JSON format with 'name' and 'arguments' keys."""

# Create a structured chat prompt template using LangChain's ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),  # System message setting the assistant's behavior
    ("user", "{input}")  # Placeholder for user input, which will be dynamically inserted at runtime
])

# This prompt template will be used to guide the LLM in correctly identifying
# and formatting tool usage based on user queries.


In [10]:
# Import ChatOpenAI from LangChain's OpenAI module
# This class provides an interface to interact with OpenAI's GPT models
from langchain_openai import ChatOpenAI

# Set the OpenAI API key using an environment variable
# getpass.getpass() securely prompts the user to enter their API key without displaying it
os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Initialize the ChatOpenAI model with specific parameters
# model="gpt-4o-mini: Specifies which GPT model to use
# temperature=0: Ensures deterministic responses (reduces randomness)
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Combine the prompt and model into a single processing chain using the | operator
# This allows user input to flow through the structured prompt and get processed by the model
chain = prompt | model

# Invoke the chain with a sample user input to test the tool
result = chain.invoke({"input": "What's 4 times 5?"})

# Print the model's response
# Expected output: A structured JSON response containing the tool name and arguments
print(result.content)



··········
{
  "name": "multiply",
  "arguments": {
    "first_int": 4,
    "second_int": 5
  }
}


In [13]:
# Import itemgetter from the operator module
# itemgetter allows extracting specific keys from a dictionary (used here for extracting "arguments")

from langchain_core.output_parsers import JsonOutputParser

from operator import itemgetter


# Create a processing chain that follows these steps:
# 1. The user input is processed by the prompt template.
# 2. The prompt is fed into the ChatOpenAI model for processing.
# 3. The model's output is parsed into JSON format using JsonOutputParser.
# 4. The parsed JSON dictionary is processed by itemgetter("arguments"), extracting only the "arguments" key.
# 5. The extracted arguments are then passed to the multiply tool to compute the final result.
chain = prompt | model | JsonOutputParser() | itemgetter("arguments") | multiply

# Invoke the full processing chain with a sample multiplication query
final_result = chain.invoke({"input": "What's 13 times 4?"})

# Print the final computed result
# Expected output: 52 (since 13 * 4 = 52)
print(final_result)


52


In [14]:
# Import the 'tool' decorator from LangChain Core
# This decorator allows defining functions as LangChain tools that can be invoked programmatically
from langchain_core.tools import tool

# Define an addition tool using the @tool decorator
@tool
def add(first_int: int, second_int: int) -> int:
    """Add two integers together."""
    return first_int + second_int  # Computes the sum of two integers

# Define an exponentiation tool using the @tool decorator
@tool
def exponentiate(base: int, exponent: int) -> int:
    """Raise the base to the exponent power."""
    return base ** exponent  # Computes the result of base raised to the given exponent


In [15]:
# Define a list of available tools
# These tools will be used by the assistant for processing user queries
tools = [add, multiply, exponentiate]

# Generate a text-based description of the available tools
# This helps the assistant understand what operations it can perform
rendered_tools = render_text_description(tools)

# Define the system prompt that instructs the assistant on how to use the tools
system_prompt = f"""You are an assistant with access to the following tools:

{rendered_tools}

Given the user query, return the tool name and input values in JSON format with 'name' and 'arguments' keys."""

# Create a structured chat prompt template using LangChain's ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),  # System message setting the assistant's behavior
    ("user", "{input}")  # Placeholder for user input, which will be dynamically inserted at runtime
])


In [16]:
# Define a function to dynamically select and execute the appropriate tool
def tool_chain(model_output):
    # Create a dictionary mapping tool names to their respective functions
    tool_map = {tool.name: tool for tool in tools}

    # Retrieve the tool function based on the model's output (tool selection)
    chosen_tool = tool_map[model_output["name"]]

    # Return a processing chain that extracts "arguments" from the model output
    # and then applies the chosen tool to those arguments
    return itemgetter("arguments") | chosen_tool

In [17]:
# Import RunnablePassthrough from LangChain Core
# This is used to pass data through a pipeline without modification,
# while also allowing additional processing or assignments.
from langchain_core.runnables import RunnablePassthrough

# Define the full processing chain
# 1. prompt - Formats the system message and user query.
# 2. model - GPT model processes the structured input.
# 3. JsonOutputParser() - Parses the model's response into JSON format.
# 4. RunnablePassthrough.assign(output=tool_chain) - Extracts the tool output dynamically.
chain = prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=tool_chain)

# Invoke the chain with a sample query asking for addition.
final_output = chain.invoke({"input": "What's 3 plus 1132?"})

# Print the final computed result
# Expected output: 1135 (since 3 + 1132 = 1135)
print(final_output)

{'name': 'add', 'arguments': {'first_int': 3, 'second_int': 1132}, 'output': 1135}


In [18]:
from langchain_core.tools import tool
from langchain.tools.render import render_text_description
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# Define tools

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    """Add two integers together."""
    return first_int + second_int

@tool
def subtract(first_int: int, second_int: int) -> int:
    """Subtract the second integer from the first integer."""
    return first_int - second_int

@tool
def divide(first_int: int, second_int: int) -> float:
    """Divide the first integer by the second integer."""
    if second_int == 0:
        raise ValueError("Division by zero is not allowed.")
    return first_int / second_int

@tool
def modulo(first_int: int, second_int: int) -> int:
    """Find the remainder when the first integer is divided by the second integer."""
    if second_int == 0:
        raise ValueError("Modulo by zero is not allowed.")
    return first_int % second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    """Raise the base to the exponent power."""
    return base ** exponent

# List of tools
tools = [add, subtract, multiply, divide, modulo, exponentiate]

# Render tool descriptions for AI understanding
rendered_tools = render_text_description(tools)

# System prompt for AI
system_prompt = f"""You are an assistant with access to the following tools:

{rendered_tools}

Given the user query, return the tool name and input values in JSON format with 'name' and 'arguments' keys."""

# Create the chat prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}")
])

# Initialize AI model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Function to select and execute the correct tool based on model output
def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

# Build the final execution chain
chain = prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=tool_chain)

# Test cases
print(chain.invoke({"input": "What is 15 divided by 3?"}))  # Expected output: 5.0
print(chain.invoke({"input": "What is 7 modulo 4?"}))        # Expected output: 3
print(chain.invoke({"input": "What is 20 minus 8?"}))        # Expected output: 12
print(chain.invoke({"input": "What is 2 to the power of 5?"})) # Expected output: 32


{'name': 'divide', 'arguments': {'first_int': 15, 'second_int': 3}, 'output': 5.0}
{'name': 'modulo', 'arguments': {'first_int': 7, 'second_int': 4}, 'output': 3}
{'name': 'subtract', 'arguments': {'first_int': 20, 'second_int': 8}, 'output': 12}
{'name': 'exponentiate', 'arguments': {'base': 2, 'exponent': 5}, 'output': 32}


In [20]:
import gradio as gr

# Function to interact with the AI-powered calculator
def calculate(user_input):
    result1 = chain.invoke({"input": user_input})
    # Extract only the result from the model output
    if isinstance(result1, dict) and 'output' in result1:
        result = result1['output']
    else:
        result = "Error: Unable to process input"

    return result

# Create Gradio interface
iface = gr.Interface(
    fn=calculate,
    inputs=gr.Textbox(placeholder="Enter a math question, e.g., 'What is 15 divided by 3?'"),
    outputs="text",
    title="AI-Powered Calculator",
    theme="Ocean",
    description="Ask me any math-related question, and I'll pick the right tool to answer it!"
)

# Launch the Gradio app
iface.launch()


/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:694: UserWarning: Cannot load Ocean. Caught Exception: The space Ocean does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>